# BeamRider con REINFORCE (GPU: CUDA o MPS)

Entrenamos un agente de Aprendizaje por Refuerzo con el algoritmo REINFORCE (con baseline) para el entorno Atari `ALE/BeamRider-v5` usando Gymnasium + ALE.

- Algoritmo: REINFORCE con baseline (actor-crítico simple), entropía y clipping de gradiente.
- Observaciones: preprocesamiento Atari (84x84, escala de grises) + apilado de 4 frames.
- Acción: espacio `Discrete(9)` propio de BeamRider.
- Hardware: GPU recomendado (CUDA en NVIDIA o MPS en Apple Silicon), entrenamiento prolongado; se guardan checkpoints para reanudar.

Referencia del entorno: [Documentación Farama ALE - BeamRider](https://ale.farama.org/environments/beam_rider/).


In [11]:
# Instalación en Colab (descomentar si es necesario)
#!pip install -q gymnasium[atari]==0.29.1 ale-py==0.10.1 torch==2.2.2 torchvision==0.17.2
#!pip install -q tqdm==4.66.4 numpy==1.26.4 matplotlib==3.8.4 imageio==2.34.1 imageio-ffmpeg==0.4.9

import sys, platform
print({'python': sys.version.split()[0], 'platform': platform.platform()})


{'python': '3.13.5', 'platform': 'macOS-15.5-arm64-arm-64bit-Mach-O'}


In [12]:
import gymnasium as gym
import gymnasium
import ale_py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.optim import Adam
from dataclasses import dataclass
from typing import List
from collections import deque
import random
import os
import cv2
import warnings
import imageio

warnings.filterwarnings('ignore')

# Registrar entornos ALE como en la plantilla
gymnasium.register_envs(ale_py)

SEED = 123
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Preferir CUDA, luego MPS (Apple Silicon), sino CPU
device = (
    torch.device('cuda') if torch.cuda.is_available() else
    (torch.device('mps') if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available() else torch.device('cpu'))
)
print('Using device:', device)

# Asegurar determinismo parcial
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


Using device: mps


In [13]:
# Grabación de video opcional de un episodio

def record_video(model: nn.Module, filename: str = 'videos/beamrider_reinforce.mp4', fps: int = 30, seed: int = SEED+2024):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    env = make_env(seed=seed, render_mode='rgb_array')
    frames = []
    obs, info = env.reset(seed=seed)
    done = False
    with torch.no_grad():
        for t in range(cfg.max_steps_per_episode):
            x = obs_to_tensor(obs)
            logits, _ = model(x)
            action = torch.argmax(F.softmax(logits, dim=-1), dim=-1).item()
            frame = env.render()
            frames.append(frame)
            obs, r, terminated, truncated, info = env.step(action)
            if terminated or truncated:
                frame = env.render()
                frames.append(frame)
                break
    env.close()
    imageio.mimwrite(filename, frames, fps=fps, quality=8)
    print('Video guardado en:', filename)

# Descomenta para grabar video de un episodio
# record_video(agent, filename='videos/beamrider_reinforce.mp4', fps=30)


In [14]:
# FrameStack mínimo compatible (sin dependencias externas)
class SimpleFrameStack(gym.Wrapper):
    def __init__(self, env, k: int = 4):
        super().__init__(env)
        self.k = k
        self.frames = deque(maxlen=k)
        obs_space = env.observation_space
        assert len(obs_space.shape) == 2 or len(obs_space.shape) == 3
        h, w = obs_space.shape[0], obs_space.shape[1]
        # Observación resultante: (H, W, K) en channel-last (para mantener compatibilidad)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(h, w, k), dtype=np.uint8)

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        self.frames.clear()
        for _ in range(self.k):
            self.frames.append(obs)
        return self._get_ob(), info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_ob(), reward, terminated, truncated, info

    def _get_ob(self):
        return np.stack(list(self.frames), axis=-1)



In [15]:
# Crear entorno Atari con preprocesamiento y frame stacking (sin AtariPreprocessing)


def make_env(seed: int = SEED, render_mode=None):
    env = gym.make('ALE/BeamRider-v5', render_mode=render_mode)
    # Convertir a escala de grises y 84x84 con OpenCV para compatibilidad amplia
    class GrayResizeWrapper(gym.ObservationWrapper):
        def __init__(self, env):
            super().__init__(env)
            h = 84
            w = 84
            self.observation_space = gym.spaces.Box(low=0, high=255, shape=(h, w), dtype=np.uint8)
        def observation(self, obs):
            # obs: uint8 (H, W, 3) RGB
            gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
            resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
            return resized
    env = GrayResizeWrapper(env)
    env = SimpleFrameStack(env, 4)
    env.reset(seed=seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env

# Conversión de observación a tensor float32 CHW en [0,1]
def obs_to_tensor(obs) -> torch.Tensor:
    # obs viene como (84,84,4), uint8 (channel-last). Convertimos a (4,84,84)
    if isinstance(obs, np.ndarray):
        arr = obs
    else:
        arr = np.array(obs)  # LazyFrames
    if arr.ndim == 3 and arr.shape[-1] == 4:
        arr = np.transpose(arr, (2, 0, 1))
    elif arr.ndim == 2:
        arr = np.stack([arr]*4, axis=0)
    tensor = torch.from_numpy(arr).float() / 255.0
    return tensor.unsqueeze(0).to(device)  # batch dimension


In [16]:
# Grabación de video opcional de un episodio

def record_video(model: nn.Module, filename: str = 'videos/beamrider_reinforce.mp4', fps: int = 30, seed: int = SEED+2024):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    env = make_env(seed=seed, render_mode='rgb_array')
    frames = []
    obs, info = env.reset(seed=seed)
    done = False
    with torch.no_grad():
        for t in range(cfg.max_steps_per_episode):
            x = obs_to_tensor(obs)
            logits, _ = model(x)
            action = torch.argmax(F.softmax(logits, dim=-1), dim=-1).item()
            frame = env.render()
            frames.append(frame)
            obs, r, terminated, truncated, info = env.step(action)
            if terminated or truncated:
                frame = env.render()
                frames.append(frame)
                break
    env.close()
    imageio.mimwrite(filename, frames, fps=fps, quality=8)
    print('Video guardado en:', filename)

# Descomenta para grabar video de un episodio
# record_video(agent, filename='videos/beamrider_reinforce.mp4', fps=30)


In [17]:
# Red Actor-Crítico CNN para Atari (REINFORCE con baseline)
class AtariActorCritic(nn.Module):
    def __init__(self, in_channels: int, n_actions: int):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU(),
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512), nn.ReLU(),
        )
        self.policy_head = nn.Linear(512, n_actions)
        self.value_head = nn.Linear(512, 1)

        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                nn.init.zeros_(m.bias)

    def forward(self, x: torch.Tensor):
        z = self.features(x)
        z = self.flatten(z)
        z = self.fc(z)
        logits = self.policy_head(z)
        value = self.value_head(z)
        return logits, value.squeeze(-1)

    def act(self, x: torch.Tensor):
        logits, value = self.forward(x)
        probs = F.softmax(logits, dim=-1)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        entropy = dist.entropy()
        return action.item(), log_prob, entropy, value


In [18]:
@dataclass
class Config:
    total_episodes: int = 3000
    max_steps_per_episode: int = 10000
    gamma: float = 0.99
    learning_rate: float = 2.5e-4
    entropy_coef: float = 0.01
    value_coef: float = 0.5
    grad_clip_norm: float = 0.5
    checkpoint_dir: str = 'checkpoints_beamrider'
    checkpoint_every_episodes: int = 50
    eval_every_episodes: int = 100
    eval_episodes: int = 10

cfg = Config()
os.makedirs(cfg.checkpoint_dir, exist_ok=True)
print(cfg)


Config(total_episodes=3000, max_steps_per_episode=10000, gamma=0.99, learning_rate=0.00025, entropy_coef=0.01, value_coef=0.5, grad_clip_norm=0.5, checkpoint_dir='checkpoints_beamrider', checkpoint_every_episodes=50, eval_every_episodes=100, eval_episodes=10)


In [19]:
# Utilidades: retorno descontado, guardar/cargar checkpoints, evaluación

def compute_returns(rewards: List[float], gamma: float) -> torch.Tensor:
    G = 0.0
    returns = []
    for r in reversed(rewards):
        G = r + gamma * G
        returns.append(G)
    returns.reverse()
    return torch.tensor(returns, dtype=torch.float32, device=device)


def save_checkpoint(model: nn.Module, optimizer: torch.optim.Optimizer, episode: int, path: str):
    torch.save({'episode': episode,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict()}, path)


def load_checkpoint(model: nn.Module, optimizer: torch.optim.Optimizer, path: str):
    ckpt = torch.load(path, map_location=device)
    model.load_state_dict(ckpt['model'])
    optimizer.load_state_dict(ckpt['optimizer'])
    return ckpt.get('episode', 0)


def evaluate(agent: nn.Module, episodes: int = 10, render: bool = False) -> float:
    env = make_env(seed=SEED + 999, render_mode='human' if render else None)
    agent.eval()
    rewards = []
    with torch.no_grad():
        for ep in range(episodes):
            obs, info = env.reset(seed=SEED + 999 + ep)
            done = False
            total_r = 0.0
            for t in range(cfg.max_steps_per_episode):
                x = obs_to_tensor(obs)
                logits, _ = agent(x)
                action = torch.argmax(F.softmax(logits, dim=-1), dim=-1).item()
                obs, r, terminated, truncated, info = env.step(action)
                total_r += float(r)
                if terminated or truncated:
                    break
            rewards.append(total_r)
    env.close()
    agent.train()
    return float(np.mean(rewards))


In [23]:
# Entrenamiento REINFORCE con baseline

env = make_env(seed=SEED, render_mode=None)
n_actions = env.action_space.n
in_channels = 4

agent = AtariActorCritic(in_channels=in_channels, n_actions=n_actions).to(device)
optimizer = Adam(agent.parameters(), lr=cfg.learning_rate)

start_episode = 0
ckpt_path = os.path.join(cfg.checkpoint_dir, 'reinforce_beamrider.pt')
if os.path.exists(ckpt_path):
    print('Cargando checkpoint desde', ckpt_path)
    start_episode = load_checkpoint(agent, optimizer, ckpt_path)

best_eval = -float('inf')

for ep in range(start_episode, cfg.total_episodes):
    obs, info = env.reset(seed=SEED + ep)
    log_probs, entropies, values, rewards = [], [], [], []
    total_reward = 0.0

    for t in range(cfg.max_steps_per_episode):
        x = obs_to_tensor(obs)
        action, log_prob, entropy, value = agent.act(x)
        obs, reward, terminated, truncated, info = env.step(action)
        log_probs.append(log_prob)
        entropies.append(entropy)
        values.append(value)
        rewards.append(float(reward))
        total_reward += float(reward)
        if terminated or truncated:
            break

    # REINFORCE con baseline: advantage = G_t - V(s_t)
    returns = compute_returns(rewards, cfg.gamma)
    values_t = torch.stack(values)
    log_probs_t = torch.stack(log_probs)
    entropies_t = torch.stack(entropies)

    advantages = returns - values_t
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    policy_loss = -(log_probs_t * advantages.detach()).mean()
    value_loss = F.mse_loss(values_t, returns)
    entropy_bonus = entropies_t.mean()

    loss = policy_loss + cfg.value_coef * value_loss - cfg.entropy_coef * entropy_bonus

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    nn.utils.clip_grad_norm_(agent.parameters(), cfg.grad_clip_norm)
    optimizer.step()

    if (ep + 1) % cfg.checkpoint_every_episodes == 0:
        save_checkpoint(agent, optimizer, ep + 1, ckpt_path)

    if (ep + 1) % cfg.eval_every_episodes == 0:
        avg_eval = evaluate(agent, episodes=cfg.eval_episodes, render=False)
        print(f'Ep {ep+1} | Reward entrenamiento: {total_reward:.1f} | Eval-10 media: {avg_eval:.1f}')
        if avg_eval > best_eval:
            best_eval = avg_eval
            torch.save({'model': agent.state_dict(), 'avg_eval': best_eval}, os.path.join(cfg.checkpoint_dir, 'best.pt'))

env.close()
print('Entrenamiento finalizado.')


Cargando checkpoint desde checkpoints_beamrider/reinforce_beamrider.pt
Ep 100 | Reward entrenamiento: 352.0 | Eval-10 media: 96.8
Ep 200 | Reward entrenamiento: 220.0 | Eval-10 media: 96.8
Ep 300 | Reward entrenamiento: 440.0 | Eval-10 media: 4.4
Ep 400 | Reward entrenamiento: 804.0 | Eval-10 media: 684.4
Ep 500 | Reward entrenamiento: 264.0 | Eval-10 media: 560.4
Ep 600 | Reward entrenamiento: 440.0 | Eval-10 media: 70.4


KeyboardInterrupt: 

In [21]:
# Grabación de video opcional de un episodio

def record_video(model: nn.Module, filename: str = 'videos/beamrider_reinforce.mp4', fps: int = 30, seed: int = SEED+2024):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    env = make_env(seed=seed, render_mode='rgb_array')
    frames = []
    obs, info = env.reset(seed=seed)
    done = False
    with torch.no_grad():
        for t in range(cfg.max_steps_per_episode):
            x = obs_to_tensor(obs)
            logits, _ = model(x)
            action = torch.argmax(F.softmax(logits, dim=-1), dim=-1).item()
            frame = env.render()
            frames.append(frame)
            obs, r, terminated, truncated, info = env.step(action)
            if terminated or truncated:
                frame = env.render()
                frames.append(frame)
                break
    env.close()
    imageio.mimwrite(filename, frames, fps=fps, quality=8)
    print('Video guardado en:', filename)

# Descomenta para grabar video de un episodio
record_video(agent, filename='videos/beamrider_reinforce.mp4', fps=30)


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Video guardado en: videos/beamrider_reinforce.mp4


In [ ]:
# Cargar mejor modelo (si existe) y evaluación en 10 episodios
best_path = os.path.join(cfg.checkpoint_dir, 'best.pt')
if os.path.exists(best_path):
    print('Cargando mejor modelo desde', best_path)
    ckpt = torch.load(best_path, map_location=device)
    agent.load_state_dict(ckpt['model'])

avg10 = evaluate(agent, episodes=10, render=False)
print(f'Recompensa promedio en 10 episodios: {avg10:.2f}')

# Grabación adicional con best.pt si existe
if os.path.exists(best_path):
    # agente temporal para evitar sobre-escribir 'agent' si estás entrenando
    env_tmp = make_env(seed=SEED+3030)
    n_actions_best = env_tmp.action_space.n
    env_tmp.close()
    best_agent = AtariActorCritic(in_channels=4, n_actions=n_actions_best).to(device)
    ckpt_best = torch.load(best_path, map_location=device)
    best_agent.load_state_dict(ckpt_best['model'])
    os.makedirs('videos', exist_ok=True)
    record_video(best_agent, filename='videos/beamrider_best.mp4', fps=30, seed=SEED+3030)
else:
    print('No se encontró best.pt; aún no hay mejor modelo guardado')

Cargando mejor modelo desde checkpoints_beamrider/best.pt
